In [85]:
# importamos librería
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime, timedelta

from sklearn.preprocessing import LabelEncoder,MinMaxScaler

import folium
import plotly.express as px

In [86]:
DATA_PATH = './data_dsmarket'

# **Dataset Eventos**

In [87]:
df_events = pd.read_csv(os.path.join(DATA_PATH,"daily_calendar_with_events.csv"))

In [88]:
df_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1913 entries, 0 to 1912
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         1913 non-null   object
 1   weekday      1913 non-null   object
 2   weekday_int  1913 non-null   int64 
 3   d            1913 non-null   object
 4   event        26 non-null     object
dtypes: int64(1), object(4)
memory usage: 74.9+ KB


In [89]:
# Change format of date column

df_events['date'] = pd.to_datetime(df_events['date'], format='%Y-%m-%d')

In [90]:
FIRST_DAY = df_events['date'].min()
FIRST_DAY

Timestamp('2011-01-29 00:00:00')

In [91]:
df_events.drop(df_events[df_events['date'] < pd.to_datetime('31-01-2011', format = '%d-%m-%Y')].index, inplace=True)

In [92]:
# Change order of columns
order_of_columns = [
  'date',
  'd',
  'weekday',
  'weekday_int',
  'event'
]
df_events = df_events[order_of_columns]

In [93]:
df_events.head(14)

,date,d,weekday,weekday_int,event
2,2011-01-31,d_3,Monday,3,NaN
3,2011-02-01,d_4,Tuesday,4,NaN
4,2011-02-02,d_5,Wednesday,5,NaN
5,2011-02-03,d_6,Thursday,6,NaN
6,2011-02-04,d_7,Friday,7,NaN
7,2011-02-05,d_8,Saturday,1,NaN
8,2011-02-06,d_9,Sunday,2,SuperBowl
9,2011-02-07,d_10,Monday,3,NaN
10,2011-02-08,d_11,Tuesday,4,NaN
11,2011-02-09,d_12,Wednesday,5,NaN


In [94]:
df_events.event.unique()

array([nan, 'SuperBowl', 'Ramadan starts', 'Thanksgiving', 'NewYear',
       'Easter'], dtype=object)

In [95]:
df_events.isnull().sum()

date              0
d                 0
weekday           0
weekday_int       0
event          1885
dtype: int64

In [96]:
df_events.fillna("None", inplace=True)

In [97]:
# One Hot Encoding of events column
df_events = pd.get_dummies(data=df_events, columns=['event'], drop_first=True, dtype='bool')

In [98]:
df_events

,date,d,weekday,weekday_int,event_NewYear,event_None,event_Ramadan starts,event_SuperBowl,event_Thanksgiving
2,2011-01-31,d_3,Monday,3,False,True,False,False,False
3,2011-02-01,d_4,Tuesday,4,False,True,False,False,False
4,2011-02-02,d_5,Wednesday,5,False,True,False,False,False
5,2011-02-03,d_6,Thursday,6,False,True,False,False,False
6,2011-02-04,d_7,Friday,7,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...
1908,2016-04-20,d_1909,Wednesday,5,False,True,False,False,False
1909,2016-04-21,d_1910,Thursday,6,False,True,False,False,False
1910,2016-04-22,d_1911,Friday,7,False,True,False,False,False
1911,2016-04-23,d_1912,Saturday,1,False,True,False,False,False


In [99]:
ramadan_start_dates = []
for _, row in df_events.iterrows():
    if row['event_Ramadan starts']:
        ramadan_start_dates.append(row['date'])

ramadan_dates = []
for date in ramadan_start_dates:
    new_date = date
    for i in range(30):
        ramadan_dates.append(new_date)
        new_date += timedelta(days=1)

for date in ramadan_dates:
    df_events.loc[df_events['date'] == date, 'event_None'] = False
    df_events.loc[df_events['date'] == date, 'event_Ramadan'] = True

In [100]:
df_events.isnull().sum()

date                       0
d                          0
weekday                    0
weekday_int                0
event_NewYear              0
event_None                 0
event_Ramadan starts       0
event_SuperBowl            0
event_Thanksgiving         0
event_Ramadan           1761
dtype: int64

In [101]:
df_events.fillna(value=False, inplace=True)

In [102]:
df_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1911 entries, 2 to 1912
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  1911 non-null   datetime64[ns]
 1   d                     1911 non-null   object        
 2   weekday               1911 non-null   object        
 3   weekday_int           1911 non-null   int64         
 4   event_NewYear         1911 non-null   bool          
 5   event_None            1911 non-null   bool          
 6   event_Ramadan starts  1911 non-null   bool          
 7   event_SuperBowl       1911 non-null   bool          
 8   event_Thanksgiving    1911 non-null   bool          
 9   event_Ramadan         1911 non-null   bool          
dtypes: bool(6), datetime64[ns](1), int64(1), object(2)
memory usage: 71.0+ KB


In [103]:
df_events.drop(columns='event_Ramadan starts', inplace=True)

In [104]:
event_columns = df_events.select_dtypes(include='bool').columns
df_events = df_events.groupby(pd.Grouper(key='date', freq='W'))[event_columns].any()

In [105]:
df_events

,event_NewYear,event_None,event_SuperBowl,event_Thanksgiving,event_Ramadan
date,,,,,
2011-02-06,False,True,True,False,False
2011-02-13,False,True,False,False,False
2011-02-20,False,True,False,False,False
2011-02-27,False,True,False,False,False
2011-03-06,False,True,False,False,False
...,...,...,...,...,...
2016-03-27,False,True,False,False,False
2016-04-03,False,True,False,False,False
2016-04-10,False,True,False,False,False


In [106]:
df_events.iloc[0].any()

True

In [107]:
def cambiar_valores(row):
    if row.any():
        return False
    else:
        return True

In [108]:
df_events['event_None'] = df_events.apply(cambiar_valores, axis=1)

In [109]:
df_events.reset_index(inplace=True)

In [110]:
#df_events.to_csv(os.path.join(DATA_PATH,"preproc_data/events_cleaned.csv"))

# **Dataset Precios**

In [111]:
df_prices = pd.read_csv(os.path.join(DATA_PATH,"item_prices.csv"))

In [112]:
df_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6965706 entries, 0 to 6965705
Data columns (total 5 columns):
 #   Column      Dtype  
---  ------      -----  
 0   item        object 
 1   category    object 
 2   store_code  object 
 3   yearweek    float64
 4   sell_price  float64
dtypes: float64(2), object(3)
memory usage: 265.7+ MB


In [113]:
df_prices.head(20)

,item,category,store_code,yearweek,sell_price
0,ACCESORIES_1_001,ACCESORIES,NYC_1,201328.0,12.7414
1,ACCESORIES_1_001,ACCESORIES,NYC_1,201329.0,12.7414
2,ACCESORIES_1_001,ACCESORIES,NYC_1,201330.0,10.9858
3,ACCESORIES_1_001,ACCESORIES,NYC_1,201331.0,10.9858
4,ACCESORIES_1_001,ACCESORIES,NYC_1,201332.0,10.9858
5,ACCESORIES_1_001,ACCESORIES,NYC_1,201333.0,10.9858
6,ACCESORIES_1_001,ACCESORIES,NYC_1,201334.0,10.9858
7,ACCESORIES_1_001,ACCESORIES,NYC_1,201335.0,10.9858
8,ACCESORIES_1_001,ACCESORIES,NYC_1,201336.0,10.9858
9,ACCESORIES_1_001,ACCESORIES,NYC_1,201337.0,10.9858


In [114]:
df_prices[df_prices['item']  == 'ACCESORIES_1_001']

,item,category,store_code,yearweek,sell_price
0,ACCESORIES_1_001,ACCESORIES,NYC_1,201328.0,12.7414
1,ACCESORIES_1_001,ACCESORIES,NYC_1,201329.0,12.7414
2,ACCESORIES_1_001,ACCESORIES,NYC_1,201330.0,10.9858
3,ACCESORIES_1_001,ACCESORIES,NYC_1,201331.0,10.9858
4,ACCESORIES_1_001,ACCESORIES,NYC_1,201332.0,10.9858
...,...,...,...,...,...
6257108,ACCESORIES_1_001,ACCESORIES,PHI_3,NaN,11.1454
6257109,ACCESORIES_1_001,ACCESORIES,PHI_3,NaN,11.1454
6257110,ACCESORIES_1_001,ACCESORIES,PHI_3,NaN,11.1454
6257111,ACCESORIES_1_001,ACCESORIES,PHI_3,NaN,11.1454


In [115]:
df_prices.isnull().sum()

item               0
category           0
store_code         0
yearweek      243920
sell_price         0
dtype: int64

In [116]:
df_prices.yearweek.min()

201105.0

In [117]:
df_prices.dropna(inplace=True)

In [118]:
# Year week to date (Stablishing Sunday for every record as it's the last day)
df_prices['yearweek'] = pd.to_datetime(
    (df_prices['yearweek'].astype(int)).astype(str) + '-0', 
    format='%Y%W-%w'
)

In [119]:
df_prices.rename(columns={'yearweek':'date'}, inplace=True)

In [120]:
df_prices.sort_values(by=['item','date', 'store_code'], inplace=True)

In [121]:
df_prices.reset_index(drop=True,inplace=True)

In [122]:
order_of_columns = [
  'item',
  'date',
  'store_code',
  'category',
  'sell_price'
]
df_prices = df_prices[order_of_columns]

In [123]:
len(df_prices['item'].unique())

3049

In [124]:
df_prices.date.min()

Timestamp('2011-02-06 00:00:00')

In [125]:
df_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6721786 entries, 0 to 6721785
Data columns (total 5 columns):
 #   Column      Dtype         
---  ------      -----         
 0   item        object        
 1   date        datetime64[ns]
 2   store_code  object        
 3   category    object        
 4   sell_price  float64       
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 256.4+ MB


In [126]:
#df_prices.to_csv(os.path.join(DATA_PATH,"preproc_data/prices_cleaned.csv"))

# **Dataset Ventas**

In [127]:
df_sales = pd.read_csv(os.path.join(DATA_PATH,"item_sales.csv"))

In [128]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1920 entries, id to d_1913
dtypes: int64(1913), object(7)
memory usage: 446.6+ MB


In [129]:
df_sales.head()

,id,item,category,department,store,store_code,region,d_1,d_2,d_3,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [130]:
len(df_sales.item.unique())

3049

In [131]:
df_sales.set_index('id', inplace=True)

In [132]:
column_names = list(df_sales.columns)
columns_to_select = column_names[8:]

In [133]:
df_sales_stack = df_sales.loc[:,columns_to_select].stack()

In [134]:
df_sales_stack = df_sales_stack.reset_index()

In [135]:
df_sales_stack.rename(columns={'level_1':'day', 0:'sales_count'}, inplace=True)

In [136]:
df_sales_stack

,id,day,sales_count
0,ACCESORIES_1_001_NYC_1,d_3,0
1,ACCESORIES_1_001_NYC_1,d_4,0
2,ACCESORIES_1_001_NYC_1,d_5,0
3,ACCESORIES_1_001_NYC_1,d_6,0
4,ACCESORIES_1_001_NYC_1,d_7,0
...,...,...,...
58266385,SUPERMARKET_3_827_PHI_3,d_1909,0
58266386,SUPERMARKET_3_827_PHI_3,d_1910,0
58266387,SUPERMARKET_3_827_PHI_3,d_1911,0
58266388,SUPERMARKET_3_827_PHI_3,d_1912,0


In [137]:
df_sales_stack['date'] = None
df_sales_stack['day'] = df_sales_stack['day'].apply(lambda x: x.split('_')[1])
df_sales_stack['date'] = FIRST_DAY +  pd.to_timedelta(df_sales_stack['day'].astype(int)-1, unit='d')

print(df_sales_stack)

                               id   day  sales_count       date
0          ACCESORIES_1_001_NYC_1     3            0 2011-01-31
1          ACCESORIES_1_001_NYC_1     4            0 2011-02-01
2          ACCESORIES_1_001_NYC_1     5            0 2011-02-02
3          ACCESORIES_1_001_NYC_1     6            0 2011-02-03
4          ACCESORIES_1_001_NYC_1     7            0 2011-02-04
...                           ...   ...          ...        ...
58266385  SUPERMARKET_3_827_PHI_3  1909            0 2016-04-20
58266386  SUPERMARKET_3_827_PHI_3  1910            0 2016-04-21
58266387  SUPERMARKET_3_827_PHI_3  1911            0 2016-04-22
58266388  SUPERMARKET_3_827_PHI_3  1912            0 2016-04-23
58266389  SUPERMARKET_3_827_PHI_3  1913            0 2016-04-24

[58266390 rows x 4 columns]


In [138]:
df_sales_stack = df_sales_stack.set_index("date").groupby(["id"]).resample('W')['sales_count'].sum().reset_index()

In [139]:
df_sales_stack.date.min()

Timestamp('2011-02-06 00:00:00')

In [140]:
column_names = list(df_sales.columns)
columns_to_select = column_names[0:6]

In [141]:
df_sales = df_sales.loc[:,columns_to_select]

In [142]:
df_sales

,item,category,department,store,store_code,region
id,,,,,,
ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York
ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York
ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York
ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York
ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York
...,...,...,...,...,...,...
SUPERMARKET_3_823_PHI_3,SUPERMARKET_3_823,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia
SUPERMARKET_3_824_PHI_3,SUPERMARKET_3_824,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia
SUPERMARKET_3_825_PHI_3,SUPERMARKET_3_825,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia


In [143]:
df_sales = df_sales.merge(df_sales_stack, on='id')

In [144]:
del(df_sales_stack)

In [145]:
df_sales.head()

,id,item,category,department,store,store_code,region,date,sales_count
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,2011-02-06,0
1,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,2011-02-13,0
2,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,2011-02-20,0
3,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,2011-02-27,0
4,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,2011-03-06,0


In [146]:
len(df_sales.index.unique())

8323770

In [147]:
len(df_sales.region.unique())

3

In [148]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8323770 entries, 0 to 8323769
Data columns (total 9 columns):
 #   Column       Dtype         
---  ------       -----         
 0   id           object        
 1   item         object        
 2   category     object        
 3   department   object        
 4   store        object        
 5   store_code   object        
 6   region       object        
 7   date         datetime64[ns]
 8   sales_count  int64         
dtypes: datetime64[ns](1), int64(1), object(7)
memory usage: 893.1+ MB


In [149]:
order_of_columns = [
  'id',
  'date',
  'item',
  'category',
  'department',
  'store_code',
  'region',
  'store',
  'sales_count'
]
df_sales_merged = df_sales[order_of_columns]

In [150]:
df_sales_merged.head()

,id,date,item,category,department,store_code,region,store,sales_count
0,ACCESORIES_1_001_NYC_1,2011-02-06,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,NYC_1,New York,Greenwich_Village,0
1,ACCESORIES_1_001_NYC_1,2011-02-13,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,NYC_1,New York,Greenwich_Village,0
2,ACCESORIES_1_001_NYC_1,2011-02-20,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,NYC_1,New York,Greenwich_Village,0
3,ACCESORIES_1_001_NYC_1,2011-02-27,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,NYC_1,New York,Greenwich_Village,0
4,ACCESORIES_1_001_NYC_1,2011-03-06,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,NYC_1,New York,Greenwich_Village,0


In [151]:
df_sales_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8323770 entries, 0 to 8323769
Data columns (total 9 columns):
 #   Column       Dtype         
---  ------       -----         
 0   id           object        
 1   date         datetime64[ns]
 2   item         object        
 3   category     object        
 4   department   object        
 5   store_code   object        
 6   region       object        
 7   store        object        
 8   sales_count  int64         
dtypes: datetime64[ns](1), int64(1), object(7)
memory usage: 893.1+ MB


In [152]:
df_sales_merged.to_csv(os.path.join(DATA_PATH,"preproc_data/sales_cleaned.csv"))

In [153]:
full_df = df_sales_merged.merge(df_events, how='left', on = 'date')

In [154]:
df_prices

,item,date,store_code,category,sell_price
0,ACCESORIES_1_001,2013-07-21,BOS_1,ACCESORIES,12.7414
1,ACCESORIES_1_001,2013-07-21,NYC_1,ACCESORIES,12.7414
2,ACCESORIES_1_001,2013-07-21,NYC_4,ACCESORIES,12.7414
3,ACCESORIES_1_001,2013-07-21,PHI_1,ACCESORIES,12.7414
4,ACCESORIES_1_001,2013-07-28,BOS_1,ACCESORIES,10.9858
...,...,...,...,...,...
6721781,SUPERMARKET_3_827,2016-05-01,NYC_3,SUPERMARKET,1.2000
6721782,SUPERMARKET_3_827,2016-05-01,NYC_4,SUPERMARKET,1.2000
6721783,SUPERMARKET_3_827,2016-05-01,PHI_1,SUPERMARKET,1.2000
6721784,SUPERMARKET_3_827,2016-05-01,PHI_2,SUPERMARKET,1.2000


In [155]:
df_prices.drop_duplicates(inplace=True)

In [156]:
full_df = full_df.merge(df_prices[['date', 'sell_price', 'item', 'store_code']], how='left', on = ['item', 'store_code', 'date'])

In [157]:
full_df.shape

(8323770, 15)

In [158]:
df_sales_merged.shape

(8323770, 9)

In [159]:
full_df['revenue'] = full_df['sell_price'] * full_df['sales_count']

In [160]:
full_df.to_csv(os.path.join(DATA_PATH,"preproc_data/full_df.csv"))